In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import theano
import theano.tensor as T

In [12]:
def init_weight(M1, M2):
    return np.random.randn(M1, M2) * np.sqrt(2.0 / M1)

class HiddenLayer(object):
    def __init__(self, M1, M2, f):
        self.M1 = M1
        self.M2 = M2
        self.f = f
        W = init_weight(M1, M2)
        b = np.zeros(M2)
        self.W = theano.shared(W)
        self.b = theano.shared(b)
        self.params = [self.W, self.b]

    def forward(self, X):
        if self.f == T.nnet.relu:
            return self.f(X.dot(self.W)+self.b, alpha=0.1)
        return self.f(X.dot(self.W)+self.b)
    
class ANN(object):
    def __init__(self, hidden_layer_sizes):
        self.hidden_layer_sizes = hidden_layer_sizes

    def fit(self, X, Y, activation=T.nnet.relu, 
            learning_rate=1e-3, mu=0.0, reg=0, 
            epochs=100, batch_sz=None, print_period=100, show_fig=True):
        X = X.astype(np.float32)
        Y = Y.astype(np.int32)

        # initialize hidden layers
        N, D = X.shape
        self.layers = []
        M1 = D
        for M2 in self.hidden_layer_sizes:
            h = HiddenLayer(M1, M2, activation)
            self.layers.append(h)
            M1 = M2
        
        # final layer
        K = len(set(Y))
        # print("K:", K)
        h = HiddenLayer(M1, K, T.nnet.softmax)
        self.layers.append(h)

        if batch_sz is None:
            batch_sz = N

        # collect params for later use
        self.params = []
        for h in self.layers:
            self.params += h.params

        # for momentum
        dparams = [theano.shared(np.zeros_like(p.get_value())) for p in self.params]

        # set up theano functions and variables
        thX = T.matrix('X')
        thY = T.ivector('Y')
        p_y_given_x = self.forward(thX)

        rcost = reg * T.mean([(p*p).sum() for p in self.params])
        cost = - T.mean(T.log(p_y_given_x[T.arange(thY.shape[0]), thY])) #+ rcost
        grads = T.grad(cost, self.params)
        prediction = T.argmax(p_y_given_x, axis=1)

        # momentum only
        updates = [
            (p, p + mu*dp - learning_rate*g) for p, dp, g in zip(self.params, dparams, grads)
        ] + [
            (dp, mu*dp - learning_rate*g) for dp, g in zip(dparams, grads)
        ]

        train_op = theano.function(
            inputs=[thX, thY],
            outputs=[cost, prediction],
            updates=updates,
        )

        self.predict_op = theano.function(
            inputs=[thX],
            outputs=prediction,
        )

        n_batches = N // batch_sz
        costs = []
        for i in range(epochs):
            if n_batches > 1:
                X, Y = shuffle(X, Y)
            for j in range(n_batches):
                Xbatch = X[j*batch_sz:(j*batch_sz+batch_sz)]
                Ybatch = Y[j*batch_sz:(j*batch_sz+batch_sz)]

                c, p = train_op(Xbatch, Ybatch)
                costs.append(c)
                if (j+1) % print_period == 0:
                    print("i:", i, "j:", j, "nb:", n_batches, "cost:", c)
        
        if show_fig:
            plt.plot(costs)
            plt.show()

    def forward(self, X):
        out = X
        for h in self.layers:
            out = h.forward(out)
        return out

    def score(self, X, Y):
        P = self.predict_op(X)
        return np.mean(Y == P)

    def predict(self, X):
        return self.predict_op(X)

In [13]:
def get_spiral():
    """
    Idea: radius -> low...high
               (don't start at 0, otherwise points will be "mushed" at origin)
           angle = low...high proportional to radius
                   [0, 2pi/6, 4pi/6, ..., 10pi/6] --> [pi/2, pi/3 + pi/2, ..., ]
     x = rcos(theta), y = rsin(theta) as usual
    """
    radius = np.linspace(1, 10, 100)
    thetas = np.empty((6, 100))
    for i in range(6):
        start_angle = np.pi*i / 3.0
        end_angle = start_angle + np.pi / 2
        points = np.linspace(start_angle, end_angle, 100)
        thetas[i] = points

    # convert into cartesian coordinates
    x1 = np.empty((6, 100))
    x2 = np.empty((6, 100))
    for i in range(6):
        x1[i] = radius * np.cos(thetas[i])
        x2[i] = radius * np.sin(thetas[i])

    # inputs
    X = np.empty((600, 2))
    X[:,0] = x1.flatten()
    X[:,1] = x2.flatten()

    # add noise
    X += np.random.randn(600, 2)*0.5

    # targets
    Y = np.array([0]*100 + [1]*100 + [0]*100 + [1]*100 + [0]*100 + [1]*100)
    return X, Y

In [14]:
X, Y = get_spiral()
X, Y = shuffle(X, Y)
Ntrain = int(0.7*len(X))
Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
Xtest, Ytest = X[Ntrain:], Y[Ntrain:]

# hyperparameters to try
hidden_layer_sizes = [
    [300],
    [100,100],
    [50,50,50],
]
learning_rates = [1e-4, 1e-3, 1e-2]
l2_penalties = [0., 0.1, 1.0]

# loop through all possible hyperparameter settings
best_validation_rate = 0
best_hls = None
best_lr = None
best_l2 = None
for hls in hidden_layer_sizes:
    for lr in learning_rates:
        for l2 in l2_penalties:
            model = ANN(hls)
            model.fit(Xtrain, Ytrain, learning_rate=lr, reg=l2, mu=0.99, epochs=3000, show_fig=False)
            validation_accuracy = model.score(Xtest, Ytest)
            train_accuracy = model.score(Xtrain, Ytrain)
            print(
                "validation_accuracy: %.3f, train_accuracy: %.3f, settings: %s, %s, %s" %
                (validation_accuracy, train_accuracy, hls, lr, l2)
            )
            if validation_accuracy > best_validation_rate:
                best_validation_rate = validation_accuracy
                best_hls = hls
                best_lr = lr
                best_l2 = l2
print("Best validation_accuracy:", best_validation_rate)
print("Best settings:")
print("hidden_layer_sizes:", best_hls)
print("learning_rate:", best_lr)
print("l2:", best_l2)

validation_accuracy: 0.606, train_accuracy: 0.714, settings: [300], 0.0001, 0.0
validation_accuracy: 0.589, train_accuracy: 0.733, settings: [300], 0.0001, 0.1
validation_accuracy: 0.578, train_accuracy: 0.714, settings: [300], 0.0001, 1.0
validation_accuracy: 0.922, train_accuracy: 0.964, settings: [300], 0.001, 0.0
validation_accuracy: 0.900, train_accuracy: 0.962, settings: [300], 0.001, 0.1
validation_accuracy: 0.928, train_accuracy: 0.976, settings: [300], 0.001, 1.0
validation_accuracy: 0.967, train_accuracy: 0.993, settings: [300], 0.01, 0.0
validation_accuracy: 0.956, train_accuracy: 0.998, settings: [300], 0.01, 0.1
validation_accuracy: 0.967, train_accuracy: 0.998, settings: [300], 0.01, 1.0
validation_accuracy: 0.772, train_accuracy: 0.833, settings: [100, 100], 0.0001, 0.0
validation_accuracy: 0.706, train_accuracy: 0.752, settings: [100, 100], 0.0001, 0.1
validation_accuracy: 0.761, train_accuracy: 0.821, settings: [100, 100], 0.0001, 1.0
validation_accuracy: 0.967, train_

# Random Search

In [15]:
X, Y = get_spiral()
X, Y = shuffle(X, Y)
Ntrain = int(0.7*len(X))
Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
Xtest, Ytest = X[Ntrain:], Y[Ntrain:]

# starting hyperparameters
M = 20
nHidden = 2
log_lr = -4
log_l2 = -2 # since we always want it to be positive
max_tries = 30
  
# loop through all possible hyperparameter settings
best_validation_rate = 0
best_hls = None
best_lr = None
best_l2 = None
for _ in range(max_tries):
    model = ANN([M]*nHidden)
    model.fit(
        Xtrain, Ytrain,
        learning_rate=10**log_lr, reg=10**log_l2,
        mu=0.99, epochs=3000, show_fig=False
    )
    validation_accuracy = model.score(Xtest, Ytest)
    train_accuracy = model.score(Xtrain, Ytrain)
    print(
        "validation_accuracy: %.3f, train_accuracy: %.3f, settings: %s, %s, %s" %
        (validation_accuracy, train_accuracy, [M]*nHidden, log_lr, log_l2)
    )
    if validation_accuracy > best_validation_rate:
        best_validation_rate = validation_accuracy
        best_M = M
        best_nHidden = nHidden
        best_lr = log_lr
        best_l2 = log_l2

    # select new hyperparams
    nHidden = best_nHidden + np.random.randint(-1, 2) # -1, 0, or 1
    nHidden = max(1, nHidden)
    M = best_M + np.random.randint(-1, 2)*10
    M = max(10, M)
    log_lr = best_lr + np.random.randint(-1, 2)
    log_l2 = best_l2 + np.random.randint(-1, 2)
print("Best validation_accuracy:", best_validation_rate)
print("Best settings:")
print("best_M:", best_M)
print("best_nHidden:", best_nHidden)
print("learning_rate:", best_lr)
print("l2:", best_l2)

validation_accuracy: 0.767, train_accuracy: 0.745, settings: [20, 20], -4, -2
validation_accuracy: 0.794, train_accuracy: 0.779, settings: [10, 10], -4, -2
validation_accuracy: 0.861, train_accuracy: 0.807, settings: [10, 10, 10], -4, -1
validation_accuracy: 0.767, train_accuracy: 0.738, settings: [10, 10, 10], -4, 0
validation_accuracy: 0.883, train_accuracy: 0.881, settings: [10, 10, 10, 10], -4, -1
validation_accuracy: 0.900, train_accuracy: 0.886, settings: [10, 10, 10, 10, 10], -4, -1
validation_accuracy: 0.722, train_accuracy: 0.695, settings: [10, 10, 10, 10, 10], -5, -2
validation_accuracy: 0.844, train_accuracy: 0.831, settings: [10, 10, 10, 10], -4, -2
validation_accuracy: 0.733, train_accuracy: 0.695, settings: [20, 20, 20, 20], -5, -2
validation_accuracy: 0.967, train_accuracy: 0.993, settings: [10, 10, 10, 10], -3, 0
validation_accuracy: 0.972, train_accuracy: 0.993, settings: [10, 10, 10, 10], -3, 0
validation_accuracy: 0.872, train_accuracy: 0.845, settings: [10, 10, 10,